A Jupyter Notebook to reproduce the figures presented in <a href="https://rokuwaki.github.io/2021EastCapeNZ/">Okuwaki et al. (2021)</a>. Use `fetchMaterials.py` and `prepBathy.py` to donwload external materials before processing.

In [1]:
import figuremods